In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from pandas import *
import csv
import numpy as np

In [10]:
ex = [[1,1,1,1,1],[2,2,2,2,2],[3,3,3,3]]
ex[0:]

[[1, 1, 1, 1, 1], [2, 2, 2, 2, 2], [3, 3, 3, 3]]

In [3]:
b = [
    [1,1,0],
    [1,1,0],
    [1,0,1]
]

m = [
    [0,1,1],
    [1,1,1],
    [0,0,1]
]

In [32]:
benign = read_csv("~/Desktop/research/alexa/benign_data/cic_output/idle.pcap_Flow.csv")  
malware = read_csv("~/Desktop/research/alexa/infected_data/cic_output/idle.pcap_Flow.csv")

benign = benign.drop(['Label', 'Src IP', 'Dst IP', 'Flow ID', 'Timestamp'], axis=1)
malware = malware.drop(['Label', 'Src IP', 'Dst IP', 'Flow ID', 'Timestamp'], axis=1)
print(benign)

      Src Port  Dst Port  Protocol  Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  \
0            0         0         0      118169498            73             1   
1         7500      7500        17      111001334             9             1   
2         2190      2190        17       61031330             1             1   
3        21302     21302        17      114482871             9             1   
4        61520      5000        17      109570318             6             1   
...        ...       ...       ...            ...           ...           ...   
2866     50326        53        17          94292             1             3   
2867     43592        53        17        5063215             3             4   
2868        80     45764         6          76959             1             2   
2869     47363        53        17          33058             1             3   
2870     38423       123        17          51777             0             2   

      TotLen Fwd Pkts  TotL

In [33]:
benign = np.array(benign)
malware = np.array(malware)
print(benign[0])

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.18169498e+08
  7.30000000e+01  1.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  6.26219128e-01  1.61876025e+06  1.04634674e+06
  6.12999600e+06  1.71000000e+02  1.16325836e+08  1.61563661e+06
  1.05334682e+06  6.12999600e+06  1.71000000e+02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  6.17756707e-01  8.46242065e-03
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

In [34]:
with open('MalwareData.csv', "w") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        
        writer.writerow(['task']+list(range(len(benign[0]))))
        
        for line in benign:
            writer.writerow(['b']+list(line))
            
        for line in malware:
            writer.writerow(['m']+list(line))


In [39]:

def kfold(clr,X,y,folds):
    auc_sum=0
    kf = StratifiedKFold(n_splits=folds)
    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clr.fit(X_train, y_train)
        pred_test = clr.predict(X_test)
        auc_sum+=metrics.accuracy_score(y_test, pred_test)
    return auc_sum/folds


def RandomForest(filePath, kfolds,estimators,numExpirments):
    
    X = read_csv(filePath)
    y=X['task']

    #changes N, and Y to 0, and 1 respectively
    s = np.unique(y)
    mapping = Series([x[0] for x in enumerate(s)], index = s)  
    y=y.map(mapping) 
    del X['task']

    #initialize random forests (by defualt it is set to 10 trees)
    rf=RandomForestClassifier(estimators)
    #run algorithm
    results = []
    for i in range(numExpirments):
        print('running expiremnt: ',i)
        results.append(kfold(rf,np.array(X),y,kfolds))
    
    return results




estimators = 100
kfolds = 10
numExpirments = 5

filePath = 'MalwareData.csv'
results = RandomForest(filePath, kfolds, estimators, numExpirments)
print(sum(results)/len(results),'+/-',np.std(results))
print(max(results),min(results))

100
running expiremnt:  0
running expiremnt:  1
running expiremnt:  2
running expiremnt:  3
running expiremnt:  4
0.9424130706752013 +/- 0.0006372714742070559
0.9432446740618362 0.9413661986714092
